In [38]:
%pip install PyMuPDF

In [39]:
from elasticsearch import Elasticsearch
import fitz
import os

In [40]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=["elastic","uxVFMV452qfPJkXeYlVW"],
    ca_certs="C:\\Users\\aonno\\Downloads\\elasticsearch-8.13.4-windows-x86_64\\elasticsearch-8.13.4\\config\\certs\\http_ca.crt"
)

es.ping()

True

In [41]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text += page.get_text()
        return text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return "Error getting the file"

In [42]:
# Function to check if a file has already been ingested
def is_file_ingested(file_name, es_client, es_index):
    query = {
        "query": {
            "term": {
                "file_name.keyword": file_name
            }
        }
    }
    res = es_client.search(index=es_index, body=query)
    return len(res['hits']['hits']) > 0

In [43]:
es_index = "pdf_documents"

In [44]:
# Function to prepare data for Elasticsearch
def prepare_data_for_elasticsearch(pdf_folder):
    data_list = []
    for root, _, files in os.walk(pdf_folder):
        for file in files:
            if file.lower().endswith('.pdf'):
                if not is_file_ingested(file, es, es_index):
                    pdf_path = os.path.join(root, file)
                    text_content = extract_text_from_pdf(pdf_path)
                    data = {
                        "file_name": file,
                        "location": pdf_path,
                        "content": text_content
                    }
                    data_list.append(data)
                else:
                    print("file already ingested.Skipping")
    return data_list


In [45]:
# Function to index data into Elasticsearch
def index_data_to_elasticsearch(data_list, es_index, es_client):
    for data in data_list:
        es_client.index(index=es_index, body=data)

In [46]:
es.ping()

True

In [47]:
pdf_folder = ".\pdf"  # Replace with the path to your PDF folder
es_host = "http://localhost:9200"  # Replace with your Elasticsearch host
es_index = "pdf_documents"  # Replace with your Elasticsearch index name

# Initialize Elasticsearch client
# es_client = Elasticsearch([es_host])
# es is es client

# Extract data from PDFs and prepare for Elasticsearch
data_list = prepare_data_for_elasticsearch(pdf_folder)

# Index data into Elasticsearch
index_data_to_elasticsearch(data_list, es_index, es)

print(f"Indexed {len(data_list)} documents into Elasticsearch index '{es_index}'.")


<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\aonno\AppData\Local\Temp\ipykernel_29940\3510188874.py:1: SyntaxWarning: invalid escape sequence '\p'
  pdf_folder = ".\pdf"  # Replace with the path to your PDF folder


file already ingested.Skipping
Error extracting text from .\pdf\Safety Policy PDF Scan Files\~$ ST Policy on Amplifying Order Safety Organisation Onboard Ships - 10 Aug 21.pdf: Failed to open file '.\\pdf\\Safety Policy PDF Scan Files\\~$ ST Policy on Amplifying Order Safety Organisation Onboard Ships - 10 Aug 21.pdf'.
Error extracting text from .\pdf\Safety Policy PDF Scan Files\Policy Guidelines Letter from all Department 2021\Hull Implementation Policies- For Safety Dept 21 Jan 21\30    H-113, POLICY ON INTEGRATED SEWAGE TREATMENT, VACUUM TOILET AND HYDROGEN SULPHIDE GAS DETECTION & ALARM SYSTEMS NCD 3930, ISSUE 3.pdf: no such file: '.\pdf\Safety Policy PDF Scan Files\Policy Guidelines Letter from all Department 2021\Hull Implementation Policies- For Safety Dept 21 Jan 21\30    H-113, POLICY ON INTEGRATED SEWAGE TREATMENT, VACUUM TOILET AND HYDROGEN SULPHIDE GAS DETECTION & ALARM SYSTEMS NCD 3930, ISSUE 3.pdf'
Error extracting text from .\pdf\Safety Policy PDF Scan Files\Policy Guid

In [48]:
# ### creta new index in elastic search
# from indexMapping import indexMapping
# try:
#     es.indices.create(index="all_products", mappings=indexMapping)
# except Exception as e:
#     print(e)

In [50]:
input_search_term = "Safety"
# print("searching...")
# print(vector_of_input_keyword)

body = {
        "query": {
            "multi_match": {
                "query": input_search_term,
                "fields": ["content", "file_name"]
            }
        },
       }
res = es.search(index=es_index, body=body)

results = res["hits"]["hits"]
print(results)


[{'_index': 'pdf_documents', '_id': 'ZuiHuI8Bs1_zjc2DYj5K', '_score': 1.078742, '_source': {'file_name': 'Institution of Safety Alert Signal.pdf', 'location': '.\\pdf\\Safety Policy PDF Scan Files\\Safety Policy Letters\\Institution of Safety Alert Signal.pdf', 'content': ' \n \n \n \n \n \n'}}, {'_index': 'pdf_documents', '_id': 'auiHuI8Bs1_zjc2DYj5q', '_score': 1.078742, '_source': {'file_name': 'Safety related orders policies SOP.pdf', 'location': '.\\pdf\\Safety Policy PDF Scan Files\\Safety Policy Letters\\Safety related orders policies SOP.pdf', 'content': ' \n \n \n \n'}}, {'_index': 'pdf_documents', '_id': 'uuiHuI8Bs1_zjc2DSz2j', '_score': 1.0707239, '_source': {'file_name': 'Conduct of Safety Audits HQSNC letter Safety 8040 6 dated 08 May 15.pdf', 'location': '.\\pdf\\Safety Policy PDF Scan Files\\Conduct of Safety Audits HQSNC letter Safety 8040 6 dated 08 May 15.pdf', 'content': ''}}, {'_index': 'pdf_documents', '_id': '5OiHuI8Bs1_zjc2DUT1_', '_score': 0.9848249, '_source': 